In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time
import pandas as pd

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Retrieve page with the requests module
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
news_title =  soup.find("div", class_="content_title").text
# print(news_title )

In [5]:
news_p = soup.find("div", class_="article_teaser_body").text
# print(news_p)

In [6]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [7]:
browser.visit(url)
time.sleep(1)
html = browser.html

In [8]:
browser.click_link_by_partial_text("FULL")
time.sleep(1)

In [9]:
browser.click_link_by_partial_text("more info")
time.sleep(1)

In [10]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [11]:
image_sec = soup.find("aside", class_="image_detail_module")

result_ptags = image_sec.find_all("p")

In [12]:
# Loop through returned results
for result in result_ptags:
    
   if (result.text.startswith("Full-Res JPG") ):
        jpg_url = result.find("a")["href"]

In [13]:
browser.click_link_by_partial_href(jpg_url)

In [14]:
featured_image_url = browser.url
# print(featured_image_url)

In [15]:
url = "https://twitter.com/marswxreport?lang=en"
response = requests.get(url)
time.sleep(1)
weather_soup = BeautifulSoup(response.text, 'html.parser')



In [16]:
# mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
result_tweets = weather_soup.find_all('p',class_="tweet-text")
# print(result_tweets)

In [17]:
# Loop through returned results
for result in result_tweets:
    
   if (result.text.startswith("Sol ") ):
        mars_weather = result.text
        break

In [18]:
# print(mars_weather)

In [19]:
url = "https://space-facts.com/mars/"

In [20]:
tables = pd.read_html(url)

In [21]:
df_facts = tables[0]

In [22]:
df_facts.columns = ["Facts","Value"]

In [23]:
df_facts.set_index("Facts",inplace =True)

In [24]:
fact_html_table = df_facts.to_html()
fact_html_table = fact_html_table.replace("\n","")

In [25]:
# fact_html_table

In [26]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [27]:
hem_link = soup.find_all("a",class_="itemLink")

In [28]:
# print(hem_link)

In [29]:
hemisphere_image_urls = []
for hem in hem_link:
    hem_dict={}
    if hem.find('h3'):
        hem_dict["title"] = hem.h3.text
#         print( hem["href"])
#         print( hem.h3.text)
        time.sleep(2)
        #browser.click_link_by_partial_href(hem["href"])
        browser.click_link_by_partial_text(hem.h3.text)
        time.sleep(2)
        html = browser.html
        soup_inner = BeautifulSoup(html, 'html.parser')
        download_div = soup_inner.find('div',class_="downloads")
#         print(download_div.find("a")["href"])
        hem_dict["img_url"] = download_div.find("a")["href"]
        browser.click_link_by_text("Back")
        time.sleep(2)
        hemisphere_image_urls.append(hem_dict)
   

In [30]:
# print(hemisphere_image_urls)

In [31]:
scrape_data = {
    "news_title" : news_title,
    "news_p" : news_p,
    "featured_image_url" : featured_image_url,
    "mars_weather" : mars_weather,
    "fact_html_table" : fact_html_table,
    "hemisphere_image_urls" : hemisphere_image_urls
}

In [32]:
scrape_data

{'fact_html_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Facts</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>',
 'featured_image_url': 'https://photojournal.jpl.nasa.gov/j

In [ ]:
 ipython nbconvert --to=python [YOUR_NOTEBOOK].ipynb